In [1]:
from datasets import load_dataset, Value, concatenate_datasets, ClassLabel, Value
import random
import pickle

d = load_dataset("jeggers/competition_math")

c:\Users\jorin\.conda\envs\reft\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
original = d

In [13]:
# filter for only numeric answers
def filter_numeric(example):
    try:
        n = float(example["extracted_solution"])
        # also check if places after decimal point is <= 4
        if n == int(n) or len(str(n).split(".")[1]) <= 4:
            return True
        return False
    except:
        return False
    
d = d.filter(filter_numeric)

# convert to float
def convert_to_float(example):
    example["extracted_solution"] = float(example["extracted_solution"])
    return example

d = d.map(convert_to_float)

Map: 100%|██████████| 3199/3199 [00:00<00:00, 5897.67 examples/s]


In [4]:
d

DatasetDict({
    train: Dataset({
        features: ['problem', 'level', 'type', 'solution', 'extracted_solution'],
        num_rows: 4867
    })
    test: Dataset({
        features: ['problem', 'level', 'type', 'solution', 'extracted_solution'],
        num_rows: 3201
    })
})

In [14]:
# print unique, to string converted answers, sorted by length of decimal part
unique_answers = set()
for example in d["train"]:
    unique_answers.add(str(example["extracted_solution"]))
for example in d["test"]:
    unique_answers.add(str(example["extracted_solution"]))
unique_answers = list(unique_answers)
unique_answers.sort(key=lambda x: len(x.split(".")[1]), reverse=True)
unique_answers

['0.4625',
 '42.1875',
 '0.625',
 '6.125',
 '0.303',
 '0.076',
 '0.175',
 '1.125',
 '0.375',
 '0.155',
 '0.088',
 '0.411',
 '0.015',
 '0.323',
 '0.039',
 '0.185',
 '1.499',
 '0.38',
 '408.04',
 '0.73',
 '8.64',
 '563.51',
 '1.17',
 '3.14',
 '0.78',
 '2.18',
 '54.55',
 '222.22',
 '8.18',
 '0.16',
 '1.21',
 '4.95',
 '6.25',
 '8.45',
 '0.24',
 '920.25',
 '1.36',
 '13.18',
 '7.78',
 '16.75',
 '0.63',
 '23.31',
 '0.57',
 '66.45',
 '0.48',
 '1.25',
 '3.21',
 '3.75',
 '3.24',
 '0.43',
 '1.23',
 '0.03',
 '1.75',
 '0.54',
 '0.79',
 '0.75',
 '-0.25',
 '0.15',
 '5.25',
 '24.75',
 '0.61',
 '1.45',
 '5.29',
 '12.25',
 '2.33',
 '8.75',
 '15.08',
 '4.14',
 '1.05',
 '0.02',
 '6.17',
 '37.72',
 '22.21',
 '18.49',
 '-1.25',
 '104.99',
 '11.25',
 '72.0',
 '891.0',
 '187.0',
 '549.0',
 '228.3',
 '49.0',
 '27648.0',
 '22.5',
 '151200.0',
 '743.0',
 '19.2',
 '83.0',
 '291.0',
 '10.0',
 '88572.0',
 '221126.0',
 '2037.0',
 '1111.0',
 '59049.0',
 '-34.0',
 '561.0',
 '353.0',
 '98752.0',
 '-8.0',
 '1221.0',
 '8

In [21]:
# create new column with null datatype
def create_new_column(example):
    example["new2"] = None
    return example

d = d.map(create_new_column)
d = d.cast_column("new2", Value("null"))
print(d["train"].features)

Casting the dataset: 100%|██████████| 3199/3199 [00:00<00:00, 26955.00 examples/s]

{'problem': Value(dtype='string', id=None), 'level': Value(dtype='string', id=None), 'type': Value(dtype='string', id=None), 'solution': Value(dtype='string', id=None), 'extracted_solution': Value(dtype='string', id=None), 'new': Value(dtype='string', id=None), 'new2': Value(dtype='null', id=None)}


In [31]:
# create column "test" with one string value (first index) and None for all other indices
d["train"] = d["train"].add_column("test", [unique_answers[0] if i == 0 else None for i in range(len(d["train"]))])

In [44]:
# print dtype of "test" column
print(d["train"].features["test"])
# perfrom train test split on d["train"]
d2 = d["train"].train_test_split(test_size=0.01)
print(d2["test"].features["test"])

Value(dtype='string', id=None)
Value(dtype='string', id=None)


In [28]:
d["train"].features["new"] == d["test"].features["new"]

True

In [19]:
# cast to string type
d = d.cast_column("new", Value("string"))
print(d["train"].features)

Casting the dataset: 100%|██████████| 3199/3199 [00:00<00:00, 30033.89 examples/s]

{'problem': Value(dtype='string', id=None), 'level': Value(dtype='string', id=None), 'type': Value(dtype='string', id=None), 'solution': Value(dtype='string', id=None), 'extracted_solution': Value(dtype='string', id=None), 'new': Value(dtype='string', id=None)}


In [ ]:
# map to empty 

In [15]:
original.push_to_hub("jeggers/competition_math", "original")
d.push_to_hub("jeggers/competition_math", "numeric")

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.63s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/jeggers/competition_math/commit/bd76e17d5b06797bc5ae48f30d0d9634c3255a86', commit_message='Upload dataset', commit_description='', oid='bd76e17d5b06797bc5ae48f30d0d9634c3255a86', pr_url=None, pr_revision=None, pr_num=None)